In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from catboost import CatBoostRegressor, Pool
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from gensim.models import Word2Vec
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import datetime
import warnings
from sklearn.metrics import mean_squared_error

In [2]:
warnings.filterwarnings('ignore')

In [6]:
#train2018 = pd.read_csv('/Users/liruizhi/Desktop/新train_水压数据_2018.csv')
#train2019 = pd.read_csv('/Users/liruizhi/Desktop/新train_水压数据_2019.csv')
#train2020 = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/test_水压数据_2020.csv')
test = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/to_predict.csv')
submit = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/submit.csv')
rawweather = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/管道压力项目/数据处理/站点1.csv')
testrawweather = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/管道压力项目/数据处理/2020站点1.csv')
#train1819 = pd.read_csv('/Users/liruizhi/Desktop/outcome.csv')
#train2020 = pd.read_csv('/Users/liruizhi/Desktop/testinput.csv')
rawweather2020 = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/管道压力项目/2020天气.csv', encoding = 'gb2312')

In [7]:
rawweather2020

,Unnamed: 0,日期,天气,wind_dir,wind_level,temp_max,temp_min
0,0,2020-01-01,多云,西南风转东北风,3-4级转3-4级,3,-2
1,1,2020-01-02,阴,东北风转北风,<3级,5,2
2,2,2020-01-03,阴转晴,北风转南风,3-4级转3-4级,8,0
3,3,2020-01-04,多云,北风转东北风,<3级,9,2
4,4,2020-01-05,雾转小雨,北风转东风,3-4级,6,4
...,...,...,...,...,...,...,...
309,309,2020-11-05,阴,南风转西南风,3-4级转3-4级,11,15
310,310,2020-11-06,晴,西风转西北风,3-4级转3-4级,7,22
311,311,2020-11-07,多云转晴,西北风转北风,3-4级,6,21
312,312,2020-11-08,晴,东北风转北风,3-4级转3-4级,3,17


In [5]:
def reshape_data(df1):
    time = df1["Time"].values
    meas = df1["MeasName"].values

    df_list = []

    for i in range(0,24):
        hour = "H"+str(i)
        pressure = df1[hour].values
        df2 = pd.DataFrame()
        df2["Time"] = time
        df2["MeasName"] = meas
        df2["Hour"] = hour
        df2["pressure"] = pressure

        df_list.append(df2)

    df3 = pd.concat(df_list)
    df3.sort_values(by = ['Time', 'MeasName'], inplace = True)
    df3 = df3.reset_index(drop=True)
    return df3

In [6]:
#train2018 = reshape_data(train2018)
#train2019 = reshape_data(train2019)
#train2020 = reshape_data(train2020)

In [7]:
#train2018['Time_time'] = pd.to_datetime(train2018['Time'])
#train2019['Time_time'] = pd.to_datetime(train2019['Time'])
#train2020['Time_time'] = pd.to_datetime(train2020['Time'])
test['Time_time'] = pd.to_datetime(test['Time'])

In [8]:
def feature1(df):
    df['Day'] = df['Time'].apply(lambda x: int(x.split('-')[-1]))
    df['Hour'] = df['Hour'].apply(lambda x: int(x.replace('H', '')))
    df['MeasName'] = df['MeasName'].apply(lambda x: int(x.replace('站点', '')))
    df['Year'] = df['Time'].apply(lambda x: int(x.split('-')[0]))
    df['Month'] = df['Time'].apply(lambda x: int(x.split('-')[1]) )

    return df

In [9]:
#train2018 = feature1(train2018)
#train2019 = feature1(train2019)
#train2020 = feature1(train2020)
test = feature1(test)

In [10]:
def feature2(df):
    df.sort_values(by=['Time', 'MeasName', 'Hour'], inplace=True)

    return df

In [11]:
#train2018 = feature2(train2018)
#train2019 = feature2(train2019)
#train2020 = feature2(train2020)

In [26]:
train1819 = pd.concat([train2018, train2019], ignore_index=True )

In [12]:
train1819

,Unnamed: 0,Time,MeasName,Hour,pressure,Time_time,Day,Year,Month,log_1_pressure,log_2_pressure,log_3_pressure
0,0,2018-01-01,1,0,0.288625,2018-01-01,1,2018,1,NaN,NaN,NaN
1,1,2018-01-01,1,1,0.292000,2018-01-01,1,2018,1,NaN,NaN,NaN
2,2,2018-01-01,1,2,0.290500,2018-01-01,1,2018,1,NaN,NaN,NaN
3,3,2018-01-01,1,3,0.299500,2018-01-01,1,2018,1,NaN,NaN,NaN
4,4,2018-01-01,1,4,0.300250,2018-01-01,1,2018,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
525595,525595,2019-12-31,30,19,0.241875,2019-12-31,31,2019,12,0.255375,0.246000,0.264000
525596,525596,2019-12-31,30,20,0.274125,2019-12-31,31,2019,12,0.255750,0.232125,0.246750
525597,525597,2019-12-31,30,21,0.261375,2019-12-31,31,2019,12,0.251625,0.223125,0.249375
525598,525598,2019-12-31,30,22,0.255000,2019-12-31,31,2019,12,0.268125,0.247500,0.232875


In [13]:
environment = pd.concat([rawweather['weather'], rawweather['temp_max'], rawweather['temp_min'], rawweather['wind_dir'], rawweather['wind_level']],axis=1)
testenvironment = pd.concat([testrawweather['天气'], testrawweather['temp_max'], testrawweather['temp_min'], testrawweather['wind_dir'], testrawweather['wind_level']],axis=1)
environment2020 = pd.concat([rawweather2020['天气'], rawweather2020['temp_max'], rawweather2020['temp_min'], rawweather2020['wind_dir'], rawweather2020['wind_level']],axis=1)
for index, row in testenvironment.iterrows():
    row['temp_max'] = row['temp_max'].strip('℃')
    row['temp_min'] = row['temp_min'].strip('℃')

In [14]:
testenvironment

,天气,temp_max,temp_min,wind_dir,wind_level
0,多云,3,-2,西南风转东北风,3-4级转3-4级
1,阴,5,2,东北风转北风,<3级
2,阴转晴,8,0,北风转南风,3-4级转3-4级
3,多云,9,2,北风转东北风,<3级
4,雾转小雨,6,4,北风转东风,3-4级
...,...,...,...,...,...
119,阴转多云,20,26,西北风转北风,3-4级转3-4级
120,多云,20,29,北风,<3级
121,晴转小雨,22,31,东北风转北风,3-4级
122,阴转小雨,24,28,东风转东北风,3-4级转3-4级


In [15]:
environment2020.shape

(314, 5)

In [16]:
def feature3(df):
    for i in range(0, 730):
        if '雨' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 730):
        if '雪' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 730):
        if '雨' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 730):
        if '雾' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 730):
        if '沙' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 730):
        if '阴' in df.iloc[i, 0]:
            df.iloc[i, 0] = '阴天'

    for i in range(0, 730):
        if '晴' in df.iloc[i, 0]:
            df.iloc[i, 0] = '晴天'

    for i in range(0, 730):
        if '多云' in df.iloc[i, 0]:
            df.iloc[i, 0] = '晴天'

    for i in range(0, 730):
        if '转东北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 730):
        if '转东南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 730):
        if '转西北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 730):
        if '转西南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 730):
        if '转东风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '东风'

    for i in range(0, 730):
        if '转西风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '西风'

    for i in range(0, 730):
        if '转南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 730):
        if '转北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 730):
        if '东北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 730):
        if '东南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 730):
        if '西北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 730):
        if '西南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 730):
        if '北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 730):
        if '南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 730):
        if '东风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '东风'

    for i in range(0, 730):
        if '西风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '西风'

    for i in range(0, 730):
        if '微风' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    for i in range(0, 730):
        if '<3级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    for i in range(0, 730):
        if '3-4级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '3-5级'

    for i in range(0, 730):
        if '4-5级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '3-5级'

    for i in range(0, 730):
        if '5-6级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '5-7级'

    for i in range(0, 730):
        if '6-7级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '5-7级'

    for i in range(0, 730):
        if '转' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    return df

def feature4(df):
    for i in range(0, 124):
        if '雨' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 124):
        if '雪' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 124):
        if '雨' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 124):
        if '雾' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 124):
        if '沙' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 124):
        if '阴' in df.iloc[i, 0]:
            df.iloc[i, 0] = '阴天'

    for i in range(0, 124):
        if '晴' in df.iloc[i, 0]:
            df.iloc[i, 0] = '晴天'

    for i in range(0, 124):
        if '多云' in df.iloc[i, 0]:
            df.iloc[i, 0] = '晴天'

    for i in range(0, 124):
        if '转东北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 124):
        if '转东南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 124):
        if '转西北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 124):
        if '转西南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 124):
        if '转东风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '东风'

    for i in range(0, 124):
        if '转西风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '西风'

    for i in range(0, 124):
        if '转南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 124):
        if '转北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 124):
        if '东北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 124):
        if '东南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 124):
        if '西北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 124):
        if '西南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 124):
        if '北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 124):
        if '南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 124):
        if '东风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '东风'

    for i in range(0, 124):
        if '西风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '西风'

    for i in range(0, 124):
        if '微风' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    for i in range(0, 124):
        if '<3级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    for i in range(0, 124):
        if '3-4级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '3-5级'

    for i in range(0, 124):
        if '4-5级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '3-5级'

    for i in range(0, 124):
        if '5-6级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '5-7级'

    for i in range(0, 124):
        if '6-7级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '5-7级'

    for i in range(0, 124):
        if '转' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    return df

def feature5(df):
    for i in range(0, 314):
        if '雨' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 314):
        if '雪' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 314):
        if '雨' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 314):
        if '雾' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 314):
        if '沙' in df.iloc[i, 0]:
            df.iloc[i, 0] = '特殊天气'

    for i in range(0, 314):
        if '阴' in df.iloc[i, 0]:
            df.iloc[i, 0] = '阴天'

    for i in range(0, 314):
        if '晴' in df.iloc[i, 0]:
            df.iloc[i, 0] = '晴天'

    for i in range(0, 314):
        if '多云' in df.iloc[i, 0]:
            df.iloc[i, 0] = '晴天'

    for i in range(0, 314):
        if '转东北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 314):
        if '转东南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 314):
        if '转西北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 314):
        if '转西南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 314):
        if '转东风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '东风'

    for i in range(0, 314):
        if '转西风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '西风'

    for i in range(0, 314):
        if '转南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 314):
        if '转北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 314):
        if '东北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 314):
        if '东南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 314):
        if '西北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 314):
        if '西南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 314):
        if '北风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '北风'

    for i in range(0, 314):
        if '南风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '南风'

    for i in range(0, 314):
        if '东风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '东风'

    for i in range(0, 314):
        if '西风' in df.iloc[i, 3]:
            df.iloc[i, 3] = '西风'

    for i in range(0, 314):
        if '微风' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    for i in range(0, 314):
        if '<3级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    for i in range(0, 314):
        if '3-4级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '3-5级'

    for i in range(0, 314):
        if '4-5级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '3-5级'

    for i in range(0, 314):
        if '5-6级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '5-7级'

    for i in range(0, 314):
        if '6-7级' in df.iloc[i, 4]:
            df.iloc[i, 4] = '5-7级'

    for i in range(0, 314):
        if '转' in df.iloc[i, 4]:
            df.iloc[i, 4] = '<3级'

    return df

In [17]:
environment = feature3(environment)
testenvironment = feature4(testenvironment)
testenvironment2020 = feature5(environment2020)

In [18]:
rgrps = ['晴天', '阴天', '特殊天气']
le = LabelEncoder()
le.fit(rgrps)
tst = environment['weather'].to_numpy().reshape(environment.shape[0],1)
le_data = le.transform(tst).reshape(environment.shape[0], 1)
environment['weather'] = environment['weather'].apply(lambda x: ['晴天', '阴天', '特殊天气'].index(x))

rgrps = ['<3级', '3-5级', '5-7级']
le = LabelEncoder()
le.fit(rgrps)
tst = environment['wind_level'].to_numpy().reshape(environment.shape[0],1)
le_data = le.transform(tst).reshape(environment.shape[0], 1)
environment['wind_level'] = environment['wind_level'].apply(lambda x: ['<3级', '3-5级', '5-7级'].index(x))

rgrps = ['东风', '西风', '南风','北风']
le = LabelEncoder()
le.fit(rgrps)
tst = environment['wind_dir'].to_numpy().reshape(environment.shape[0],1)
le_data = le.transform(tst).reshape(environment.shape[0], 1)
environment['wind_dir'] = environment['wind_dir'].apply(lambda x: ['东风', '西风', '南风','北风'].index(x))

In [19]:
rgrps = ['晴天', '阴天', '特殊天气']
le = LabelEncoder()
le.fit(rgrps)
tst = testenvironment['天气'].to_numpy().reshape(testenvironment.shape[0],1)
le_data = le.transform(tst).reshape(testenvironment.shape[0], 1)
testenvironment['天气'] = testenvironment['天气'].apply(lambda x: ['晴天', '阴天', '特殊天气'].index(x))

rgrps = ['<3级', '3-5级', '5-7级']
le = LabelEncoder()
le.fit(rgrps)
tst = testenvironment['wind_level'].to_numpy().reshape(testenvironment.shape[0],1)
le_data = le.transform(tst).reshape(testenvironment.shape[0], 1)
testenvironment['wind_level'] = testenvironment['wind_level'].apply(lambda x: ['<3级', '3-5级', '5-7级'].index(x))


rgrps = ['东风', '西风', '南风','北风']
le = LabelEncoder()
le.fit(rgrps)
tst = testenvironment['wind_dir'].to_numpy().reshape(testenvironment.shape[0],1)
le_data = le.transform(tst).reshape(testenvironment.shape[0], 1)
testenvironment['wind_dir'] = testenvironment['wind_dir'].apply(lambda x: ['东风', '西风', '南风','北风'].index(x))
testenvironment.rename(columns={'天气': 'weather'}, inplace=True)
# print('SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSo cool')
print(testenvironment.head(5))

SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSo cool
   weather temp_max temp_min  wind_dir  wind_level
0        0        3       -2         3           1
1        1        5        2         3           0
2        1        8        0         2           1
3        0        9        2         3           0
4        2        6        4         0           1


In [20]:
rgrps = ['晴天', '阴天', '特殊天气']
le = LabelEncoder()
le.fit(rgrps)
tst = testenvironment2020['天气'].to_numpy().reshape(testenvironment2020.shape[0],1)
le_data = le.transform(tst).reshape(testenvironment2020.shape[0], 1)
testenvironment2020['天气'] = testenvironment2020['天气'].apply(lambda x: ['晴天', '阴天', '特殊天气'].index(x))

rgrps = ['<3级', '3-5级', '5-7级']
le = LabelEncoder()
le.fit(rgrps)
tst = testenvironment2020['wind_level'].to_numpy().reshape(testenvironment2020.shape[0],1)
le_data = le.transform(tst).reshape(testenvironment2020.shape[0], 1)
testenvironment2020['wind_level'] = testenvironment2020['wind_level'].apply(lambda x: ['<3级', '3-5级', '5-7级'].index(x))


rgrps = ['东风', '西风', '南风','北风']
le = LabelEncoder()
le.fit(rgrps)
tst = testenvironment2020['wind_dir'].to_numpy().reshape(testenvironment2020.shape[0],1)
le_data = le.transform(tst).reshape(testenvironment2020.shape[0], 1)
testenvironment2020['wind_dir'] = testenvironment2020['wind_dir'].apply(lambda x: ['东风', '西风', '南风','北风'].index(x))
testenvironment2020.rename(columns={'天气': 'weather'}, inplace=True)
# print('SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSo cool')
print(testenvironment2020.head(5))

SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSo cool
   weather  temp_max  temp_min  wind_dir  wind_level
0        0         3        -2         3           1
1        1         5         2         3           0
2        1         8         0         2           1
3        0         9         2         3           0
4        2         6         4         0           1


In [21]:
rawweather['Time_time'] = pd.to_datetime(rawweather['Time'])
testrawweather['Time_time'] = pd.to_datetime(testrawweather['Time'])
rawweather2020['Time_time'] = pd.to_datetime(rawweather2020['日期'])
my_newenvironment = pd.concat([rawweather['Time_time'], environment],axis=1)
print(my_newenvironment)

my_newenvironment2020 = pd.concat([testrawweather['Time_time'], testenvironment],axis=1)
print(my_newenvironment2020)

my_newenvironment2020whole = pd.concat([rawweather2020['Time_time'], testenvironment2020],axis=1)
print(my_newenvironment2020whole)

     Time_time  weather  temp_max  temp_min  wind_dir  wind_level
0   2018-01-01        0         1        -4         2           0
1   2018-01-02        1         8         0         3           1
2   2018-01-03        2         1        -1         3           1
3   2018-01-04        1         0        -4         3           1
4   2018-01-05        1         3        -4         3           1
..         ...      ...       ...       ...       ...         ...
725 2019-12-27        0         6        -1         2           1
726 2019-12-28        2        10         4         2           1
727 2019-12-29        0        11         2         3           0
728 2019-12-30        1         4        -6         3           1
729 2019-12-31        0         0        -5         2           0

[730 rows x 6 columns]
     Time_time  weather temp_max temp_min  wind_dir  wind_level
0   2020-01-01        0        3       -2         3           1
1   2020-01-02        1        5        2         3     

In [28]:
for index, row in train1819.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-1)
    if log_1_time >= train1819.loc[0, 'Time_time']:
        buf = train1819.loc[(train1819['Time_time'] == log_1_time) & (train1819["MeasName"]==mea) & (train1819['Hour'] == hour), 'pressure']
        train1819.loc[index, 'log_1_pressure'] = buf.values[0]
    log_2_time = time + datetime.timedelta(days=-2)
    if log_2_time >= train1819.loc[0, 'Time_time']:
        buf = train1819.loc[(train1819['Time_time'] == log_2_time) & (train1819["MeasName"]==mea) & (train1819['Hour'] == hour), 'pressure']
        train1819.loc[index, 'log_2_pressure'] = buf.values[0]
    log_3_time = time + datetime.timedelta(days=-3)
    if log_3_time >= train1819.loc[0, 'Time_time']:
        buf = train1819.loc[(train1819['Time_time'] == log_3_time) & (train1819["MeasName"]==mea) & (train1819['Hour'] == hour), 'pressure']
        train1819.loc[index, 'log_3_pressure'] = buf.values[0]
        

#for index, row in train1819.iterrows():
    #time = row['Time_time']
    #hour = row['Hour']
    #mea = row['MeasName']
    #log_1_time = time + datetime.timedelta(days=-2)
    #if log_1_time >= train1819.loc[0, 'Time_time']:
        #buf = train1819.loc[(train1819['Time_time'] == log_1_time) & (train1819["MeasName"]==mea) & (train1819['Hour'] == hour), 'pressure']
        #train1819.loc[index, 'log_2_pressure'] = buf.values[0]
        

#for index, row in train1819.iterrows():
    #time = row['Time_time']
    #hour = row['Hour']
    #mea = row['MeasName']
    #log_1_time = time + datetime.timedelta(days=-3)
    #if log_1_time >= train1819.loc[0, 'Time_time']:
        #buf = train1819.loc[(train1819['Time_time'] == log_1_time) & (train1819["MeasName"]==mea) & (train1819['Hour'] == hour), 'pressure']
        #train1819.loc[index, 'log_3_pressure'] = buf.values[0]

In [29]:
train1819

,Time,MeasName,Hour,pressure,Time_time,Day,Year,Month,log_1_pressure,log_2_pressure,log_3_pressure
0,2018-01-01,1,0,0.288625,2018-01-01,1,2018,1,NaN,NaN,NaN
1,2018-01-01,1,1,0.292000,2018-01-01,1,2018,1,NaN,NaN,NaN
2,2018-01-01,1,2,0.290500,2018-01-01,1,2018,1,NaN,NaN,NaN
3,2018-01-01,1,3,0.299500,2018-01-01,1,2018,1,NaN,NaN,NaN
4,2018-01-01,1,4,0.300250,2018-01-01,1,2018,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
525595,2019-12-31,30,19,0.241875,2019-12-31,31,2019,12,0.255375,0.246000,0.264000
525596,2019-12-31,30,20,0.274125,2019-12-31,31,2019,12,0.255750,0.232125,0.246750
525597,2019-12-31,30,21,0.261375,2019-12-31,31,2019,12,0.251625,0.223125,0.249375
525598,2019-12-31,30,22,0.255000,2019-12-31,31,2019,12,0.268125,0.247500,0.232875


In [30]:
for index, row in train2020.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-1)
    if log_1_time >= train2020.loc[0, 'Time_time']:
        buf = train2020.loc[(train2020['Time_time'] == log_1_time) & (train2020["MeasName"]==mea) & (train2020['Hour'] == hour), 'pressure']
        train2020.loc[index, 'log_1_pressure'] = buf.values[0]
    log_2_time = time + datetime.timedelta(days=-2)
    if log_2_time >= train2020.loc[0, 'Time_time']:
        buf = train2020.loc[(train2020['Time_time'] == log_2_time) & (train2020["MeasName"]==mea) & (train2020['Hour'] == hour), 'pressure']
        train2020.loc[index, 'log_2_pressure'] = buf.values[0]
    log_3_time = time + datetime.timedelta(days=-3)
    if log_3_time >= train2020.loc[0, 'Time_time']:
        buf = train2020.loc[(train2020['Time_time'] == log_3_time) & (train2020["MeasName"]==mea) & (train2020['Hour'] == hour), 'pressure']
        train2020.loc[index, 'log_3_pressure'] = buf.values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [22]:
train2020

,Unnamed: 0,Time,MeasName,Hour,pressure,Time_time,Day,Year,Month,log_1_pressure,log_2_pressure,log_3_pressure
0,0,2020-01-01,1,0,0.309625,2020-01-01,1,2020,1,NaN,NaN,NaN
1,1,2020-01-01,1,1,0.325750,2020-01-01,1,2020,1,NaN,NaN,NaN
2,2,2020-01-01,1,2,0.332875,2020-01-01,1,2020,1,NaN,NaN,NaN
3,3,2020-01-01,1,3,0.324250,2020-01-01,1,2020,1,NaN,NaN,NaN
4,4,2020-01-01,1,4,0.331750,2020-01-01,1,2020,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89275,89131,2020-08-31,30,19,0.282375,2020-08-31,31,2020,8,0.295500,0.306750,0.309375
89276,89132,2020-08-31,30,20,0.274875,2020-08-31,31,2020,8,0.288750,0.306375,0.301500
89277,89133,2020-08-31,30,21,0.244500,2020-08-31,31,2020,8,0.273000,0.281625,0.286500
89278,89134,2020-08-31,30,22,0.260250,2020-08-31,31,2020,8,0.284250,0.269625,0.268500


In [25]:
for index, row in train2018.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-1)
    if log_1_time >= train2018.loc[0, 'Time_time']:
        buf = train2018.loc[(train2018['Time_time'] == log_1_time) & (train2018["MeasName"]==mea) & (train2018['Hour'] == hour), 'pressure']
        train2018.loc[index, 'log_1_pressure'] = buf.values[0]
        

for index, row in train2018.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-2)
    if log_1_time >= train2018.loc[0, 'Time_time']:
        buf = train2018.loc[(train2018['Time_time'] == log_1_time) & (train2018["MeasName"]==mea) & (train2018['Hour'] == hour), 'pressure']
        train2018.loc[index, 'log_2_pressure'] = buf.values[0]
        

for index, row in train2018.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-3)
    if log_1_time >= train2018.loc[0, 'Time_time']:
        buf = train2018.loc[(train2018['Time_time'] == log_1_time) & (train2018["MeasName"]==mea) & (train2018['Hour'] == hour), 'pressure']
        train2018.loc[index, 'log_3_pressure'] = buf.values[0]

KeyboardInterrupt: 

In [353]:
for index, row in train2019.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-1)
    if log_1_time >= train2019.loc[0, 'Time_time']:
        buf = train2019.loc[(train2019['Time_time'] == log_1_time) & (train2019["MeasName"]==mea) & (train2019['Hour'] == hour), 'pressure']
        train2019.loc[index, 'log_1_pressure'] = buf.values[0]
        

for index, row in train2019.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-2)
    if log_1_time >= train2019.loc[0, 'Time_time']:
        buf = train2019.loc[(train2019['Time_time'] == log_1_time) & (train2019["MeasName"]==mea) & (train2019['Hour'] == hour), 'pressure']
        train2019.loc[index, 'log_2_pressure'] = buf.values[0]
        

for index, row in train2019.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-3)
    if log_1_time >= train2019.loc[0, 'Time_time']:
        buf = train2019.loc[(train2019['Time_time'] == log_1_time) & (train2019["MeasName"]==mea) & (train2019['Hour'] == hour), 'pressure']
        train2019.loc[index, 'log_3_pressure'] = buf.values[0]

In [359]:
print(train2019)

              Time  MeasName  Hour  pressure  Time_time  Day  Year  Month  \
0       2019-01-01         1     0  0.225625 2019-01-01    1  2019      1   
1       2019-01-01         1     1  0.246250 2019-01-01    1  2019      1   
2       2019-01-01         1     2  0.259000 2019-01-01    1  2019      1   
3       2019-01-01         1     3  0.251875 2019-01-01    1  2019      1   
4       2019-01-01         1     4  0.265750 2019-01-01    1  2019      1   
...            ...       ...   ...       ...        ...  ...   ...    ...   
262651  2019-12-31        30    19  0.241875 2019-12-31   31  2019     12   
262652  2019-12-31        30    20  0.274125 2019-12-31   31  2019     12   
262653  2019-12-31        30    21  0.261375 2019-12-31   31  2019     12   
262654  2019-12-31        30    22  0.255000 2019-12-31   31  2019     12   
262655  2019-12-31        30    23  0.274875 2019-12-31   31  2019     12   

        log_1_pressure  log_2_pressure  log_3_pressure  
0                 

In [44]:
#rawweather['Time_time'] = pd.to_datetime(rawweather['Time'])
#testrawweather['Time_time'] = pd.to_datetime(testrawweather['Time'])
#my_newenvironment = pd.concat([rawweather['Time_time'], my_environment],axis=1)
print(my_newenvironment)

#my_newenvironment2020 = pd.concat([testrawweather['Time_time'], my_environment2020],axis=1)
print(my_newenvironment2020)

     Time_time  weather  temp_max  temp_min  wind_dir  wind_level
0   2018-01-01        0         1        -4         2           0
1   2018-01-02        1         8         0         3           1
2   2018-01-03        2         1        -1         3           1
3   2018-01-04        1         0        -4         3           1
4   2018-01-05        1         3        -4         3           1
..         ...      ...       ...       ...       ...         ...
725 2019-12-27        0         6        -1         2           1
726 2019-12-28        2        10         4         2           1
727 2019-12-29        0        11         2         3           0
728 2019-12-30        1         4        -6         3           1
729 2019-12-31        0         0        -5         2           0

[730 rows x 6 columns]
     Time_time  weather temp_max temp_min  wind_dir  wind_level
0   2020-01-01        0        3       -2         3           1
1   2020-01-02        1        5        2         3     

In [32]:
for index, row in train2020.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-1)
    if log_1_time >= train2020.loc[0, 'Time_time']:
        buf = train2020.loc[(train2020['Time_time'] == log_1_time) & (train2020["MeasName"]==mea) & (train2020['Hour'] == hour), 'pressure']
        if(len(buf)):
            train2020.loc[index, 'log_1_pressure'] = buf.values[0]
        
for index, row in train2020.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-2)
    if log_1_time >= train2020.loc[0, 'Time_time']:
        buf = train2020.loc[(train2020['Time_time'] == log_1_time) & (train2020["MeasName"]==mea) & (train2020['Hour'] == hour), 'pressure']
        if(len(buf)):
            train2020.loc[index, 'log_2_pressure'] = buf.values[0]       

for index, row in train2020.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-3)
    if log_1_time >= train2020.loc[0, 'Time_time']:
        buf = train2020.loc[(train2020['Time_time'] == log_1_time) & (train2020["MeasName"]==mea) & (train2020['Hour'] == hour), 'pressure']
        if(len(buf)):
            train2020.loc[index, 'log_3_pressure'] = buf.values[0]

print(train2020.head(60))
print(train2020.tail(60))
print(train2020)

           Time  MeasName  Hour  pressure  Time_time  Day  Year  Month  \
0    2020-01-01         1     0  0.309625 2020-01-01    1  2020      1   
1    2020-01-01         1     1  0.325750 2020-01-01    1  2020      1   
2    2020-01-01         1     2  0.332875 2020-01-01    1  2020      1   
3    2020-01-01         1     3  0.324250 2020-01-01    1  2020      1   
4    2020-01-01         1     4  0.331750 2020-01-01    1  2020      1   
5    2020-01-01         1     5  0.327250 2020-01-01    1  2020      1   
6    2020-01-01         1     6  0.315250 2020-01-01    1  2020      1   
7    2020-01-01         1     7  0.305875 2020-01-01    1  2020      1   
8    2020-01-01         1     8  0.292375 2020-01-01    1  2020      1   
9    2020-01-01         1     9  0.269500 2020-01-01    1  2020      1   
10   2020-01-01         1    10  0.272500 2020-01-01    1  2020      1   
11   2020-01-01         1    11  0.274000 2020-01-01    1  2020      1   
12   2020-01-01         1    12  0.283

In [26]:
train2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89280 entries, 0 to 89279
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      89280 non-null  int64  
 1   Time            89280 non-null  object 
 2   MeasName        89280 non-null  int64  
 3   Hour            89280 non-null  int64  
 4   pressure        89280 non-null  float64
 5   Time_time       89280 non-null  object 
 6   Day             89280 non-null  int64  
 7   Year            89280 non-null  int64  
 8   Month           89280 non-null  int64  
 9   log_1_pressure  86400 non-null  float64
 10  log_2_pressure  83520 non-null  float64
 11  log_3_pressure  80640 non-null  float64
dtypes: float64(4), int64(6), object(2)
memory usage: 8.2+ MB


In [445]:
weatherrelated2018 = my_newenvironment.loc[0:364, :]
weatherrelated2019 = my_newenvironment.loc[365:729, :]
print(weatherrelated2019)

     Time_time  weather  temp_max  temp_min  wind_dir  wind_level
365 2019-01-01        0         2        -5         3           1
366 2019-01-02        0         3        -4         2           1
367 2019-01-03        2         5        -2         2           1
368 2019-01-04        2         0        -1         3           1
369 2019-01-05        1         0        -5         3           1
..         ...      ...       ...       ...       ...         ...
725 2019-12-27        0         6        -1         2           1
726 2019-12-28        2        10         4         2           1
727 2019-12-29        0        11         2         3           0
728 2019-12-30        1         4        -6         3           1
729 2019-12-31        0         0        -5         2           0

[365 rows x 6 columns]


In [16]:
my_newenvironment2020

,Time_time,weather,temp_max,temp_min,wind_dir,wind_level
0,2020-01-01,0,3,-2,3,1
1,2020-01-02,1,5,2,3,0
2,2020-01-03,1,8,0,2,1
3,2020-01-04,0,9,2,3,0
4,2020-01-05,2,6,4,0,1
...,...,...,...,...,...,...
119,2020-08-27,1,20,26,3,1
120,2020-08-28,0,20,29,3,0
121,2020-08-29,2,22,31,3,1
122,2020-08-30,2,24,28,3,1


In [18]:
train2020

,Unnamed: 0,Time,MeasName,Hour,pressure,Time_time,Day,Year,Month,log_1_pressure,log_2_pressure,log_3_pressure
0,0,2020-01-01,1,0,0.309625,2020-01-01,1,2020,1,NaN,NaN,NaN
1,1,2020-01-01,1,1,0.325750,2020-01-01,1,2020,1,NaN,NaN,NaN
2,2,2020-01-01,1,2,0.332875,2020-01-01,1,2020,1,NaN,NaN,NaN
3,3,2020-01-01,1,3,0.324250,2020-01-01,1,2020,1,NaN,NaN,NaN
4,4,2020-01-01,1,4,0.331750,2020-01-01,1,2020,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89275,89131,2020-08-31,30,19,0.282375,2020-08-31,31,2020,8,0.295500,0.306750,0.309375
89276,89132,2020-08-31,30,20,0.274875,2020-08-31,31,2020,8,0.288750,0.306375,0.301500
89277,89133,2020-08-31,30,21,0.244500,2020-08-31,31,2020,8,0.273000,0.281625,0.286500
89278,89134,2020-08-31,30,22,0.260250,2020-08-31,31,2020,8,0.284250,0.269625,0.268500


In [23]:
train2020['Time_time'] = pd.to_datetime(train2020['Time'])
train1819['Time_time'] = pd.to_datetime(train1819['Time'])
#result = pd.merge(train2018, weatherrelated2018, on='Time_time')
result2 = pd.merge(train2020, my_newenvironment2020, on='Time_time')
result = pd.merge(train1819, my_newenvironment, on='Time_time')
testresult = pd.merge(test, my_newenvironment2020whole, on='Time_time')
#result4 = pd.concat([result, result3])
print('result 为：')
print(result)
print('Result 2为：')
print(result2)
print('testresult为:')
print(testresult)
#print('Result 4为:')
#print(result4)
#print(train2018)


result 为：
        Unnamed: 0        Time  MeasName  Hour  pressure  Time_time  Day  \
0                0  2018-01-01         1     0  0.288625 2018-01-01    1   
1                1  2018-01-01         1     1  0.292000 2018-01-01    1   
2                2  2018-01-01         1     2  0.290500 2018-01-01    1   
3                3  2018-01-01         1     3  0.299500 2018-01-01    1   
4                4  2018-01-01         1     4  0.300250 2018-01-01    1   
...            ...         ...       ...   ...       ...        ...  ...   
525595      525595  2019-12-31        30    19  0.241875 2019-12-31   31   
525596      525596  2019-12-31        30    20  0.274125 2019-12-31   31   
525597      525597  2019-12-31        30    21  0.261375 2019-12-31   31   
525598      525598  2019-12-31        30    22  0.255000 2019-12-31   31   
525599      525599  2019-12-31        30    23  0.274875 2019-12-31   31   

        Year  Month  log_1_pressure  log_2_pressure  log_3_pressure  weather 

In [28]:
result181920 = pd.concat([result, result2], ignore_index= True)
result181920.isnull().sum()

Unnamed: 0            0
Time                  0
MeasName              0
Hour                  0
pressure              0
Time_time             0
Day                   0
Year                  0
Month                 0
log_1_pressure     3600
log_2_pressure     7200
log_3_pressure    10800
weather               0
temp_max              0
temp_min              0
wind_dir              0
wind_level            0
dtype: int64

In [29]:
for index, row in result181920.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-1)
    if log_1_time >= result181920.loc[0, 'Time_time']:
        buf = result181920.loc[(result181920['Time_time'] == log_1_time) & (result181920["MeasName"] == mea) & (
                    result181920['Hour'] == hour), 'pressure']
        if (len(buf)):
            result181920.loc[index, 'log_1_pressure'] = buf.values[0]

for index, row in result181920.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-2)
    if log_1_time >= result181920.loc[0, 'Time_time']:
        buf = result181920.loc[(result181920['Time_time'] == log_1_time) & (result181920["MeasName"] == mea) & (
                    result181920['Hour'] == hour), 'pressure']
        if (len(buf)):
            result181920.loc[index, 'log_2_pressure'] = buf.values[0]

for index, row in result181920.iterrows():
    time = row['Time_time']
    hour = row['Hour']
    mea = row['MeasName']
    log_1_time = time + datetime.timedelta(days=-3)
    if log_1_time >= result181920.loc[0, 'Time_time']:
        buf = result181920.loc[(result181920['Time_time'] == log_1_time) & (result181920["MeasName"] == mea) & (
                    result181920['Hour'] == hour), 'pressure']
        if (len(buf)):
            result181920.loc[index, 'log_3_pressure'] = buf.values[0]

In [32]:
result181920.isnull().sum()

Unnamed: 0           0
Time                 0
MeasName             0
Hour                 0
pressure             0
Time_time            0
Day                  0
Year                 0
Month                0
log_1_pressure    2880
log_2_pressure    5760
log_3_pressure    8640
weather              0
temp_max             0
temp_min             0
wind_dir             0
wind_level           0
dtype: int64

In [103]:
test

,id,Time,MeasName,Hour,Time_time,Day,Year,Month
0,0,2020-02-03,4,0,2020-02-03,3,2020,2
1,1,2020-02-03,4,1,2020-02-03,3,2020,2
2,2,2020-02-03,4,2,2020-02-03,3,2020,2
3,3,2020-02-03,4,3,2020-02-03,3,2020,2
4,4,2020-02-03,4,4,2020-02-03,3,2020,2
...,...,...,...,...,...,...,...,...
40315,40315,2020-09-20,6,19,2020-09-20,20,2020,9
40316,40316,2020-09-20,6,20,2020-09-20,20,2020,9
40317,40317,2020-09-20,6,21,2020-09-20,20,2020,9
40318,40318,2020-09-20,6,22,2020-09-20,20,2020,9


In [22]:
#result4.dropna(axis=0,how='any')
#print(result['pressure'])
#print(result4)

In [23]:
train2018Mon2 = train2018[(train2018['Time_time'] >= '2018-2-1') & (train2018['Time_time'] <= '2018-2-28')]
train2018Mon1 = train2018[(train2018['Time_time'] >= '2018-1-1') & (train2018['Time_time'] <= '2018-1-28')]
Mon_2_1_2018 = train2018Mon2['pressure'].mean() - train2018Mon1['pressure'].mean()
print(Mon_2_1_2018)

NameError: name 'train2018' is not defined

In [24]:
print(train2020.head())

   Unnamed: 0        Time  MeasName  Hour  pressure   Time_time  Day  Year  \
0           0  2020-01-01         1     0  0.309625  2020-01-01    1  2020   
1           1  2020-01-01         1     1  0.325750  2020-01-01    1  2020   
2           2  2020-01-01         1     2  0.332875  2020-01-01    1  2020   
3           3  2020-01-01         1     3  0.324250  2020-01-01    1  2020   
4           4  2020-01-01         1     4  0.331750  2020-01-01    1  2020   

   Month  log_1_pressure  log_2_pressure  log_3_pressure  
0      1             NaN             NaN             NaN  
1      1             NaN             NaN             NaN  
2      1             NaN             NaN             NaN  
3      1             NaN             NaN             NaN  
4      1             NaN             NaN             NaN  


In [105]:
train1 = result2[(result2['Time_time'] >= '2020-8-1') & (result2['Time_time'] <= '2020-8-31')]
#test1 = test[(test['Time_time'] >= '2020-2-3') & (test['Time_time'] <= '2020-2-16')]
#train2 = result[(result['Time_time'] >= '2018-8-1') & (result['Time_time'] <= '2018-8-31')]


used_feat = [f for f in result2.columns if f not in ['id', 'pressure', 'Time', 'Time_time']]
print('feat nums ', len(used_feat), used_feat)

feat nums  14 ['Unnamed: 0', 'MeasName', 'Hour', 'Day', 'Year', 'Month', 'log_1_pressure', 'log_2_pressure', 'log_3_pressure', 'weather', 'temp_max', 'temp_min', 'wind_dir', 'wind_level']


In [333]:
#print(result4[used_feat])
print(train2020)
#print(train1[used_feat])
#print()
#print(train2['pressure'])


             Time  MeasName  Hour  pressure  Time_time  Day  Year  Month  \
0      2020-01-01         1     0  0.309625 2020-01-01    1  2020      1   
1      2020-01-01         1     1  0.325750 2020-01-01    1  2020      1   
2      2020-01-01         1     2  0.332875 2020-01-01    1  2020      1   
3      2020-01-01         1     3  0.324250 2020-01-01    1  2020      1   
4      2020-01-01         1     4  0.331750 2020-01-01    1  2020      1   
...           ...       ...   ...       ...        ...  ...   ...    ...   
89131  2020-08-31        30    19  0.282375 2020-08-31   31  2020      8   
89132  2020-08-31        30    20  0.274875 2020-08-31   31  2020      8   
89133  2020-08-31        30    21  0.244500 2020-08-31   31  2020      8   
89134  2020-08-31        30    22  0.260250 2020-08-31   31  2020      8   
89135  2020-08-31        30    23  0.318000 2020-08-31   31  2020      8   

       log_1_pressure  log_2_pressure  log_3_pressure  
0                 NaN          

In [106]:
NNN = result
print(NNN[used_feat].describe())
print(NNN[used_feat].shape)
print(NNN[used_feat].isnull().sum())
print(NNN[used_feat].info())
NNN.dropna(subset=['log_3_pressure'], inplace=True)
print(NNN[used_feat].describe())
print(NNN[used_feat].shape)
print(NNN[used_feat].isnull().sum())
print(NNN[used_feat].info())

         Unnamed: 0      MeasName           Hour            Day      Year  \
count  525600.00000  525600.00000  525600.000000  525600.000000  525600.0   
mean   262799.50000      15.50000      11.500000      15.720548    2018.5   
std    151727.79508       8.65545       6.922193       8.796255       0.5   
min         0.00000       1.00000       0.000000       1.000000    2018.0   
25%    131399.75000       8.00000       5.750000       8.000000    2018.0   
50%    262799.50000      15.50000      11.500000      16.000000    2018.5   
75%    394199.25000      23.00000      17.250000      23.000000    2019.0   
max    525599.00000      30.00000      23.000000      31.000000    2019.0   

               Month  log_1_pressure  log_2_pressure  log_3_pressure  \
count  525600.000000   524880.000000   524160.000000   523440.000000   
mean        6.526027        0.250933        0.250900        0.250879   
std         3.447855        0.062097        0.062082        0.062067   
min         1.0000

In [107]:
train_data = NNN[used_feat]
train_labels = NNN['pressure']
eval_data = train1[used_feat]
eval_labels = train1['pressure']

model = CatBoostRegressor(iterations=2000, depth=10)
model.fit(train_data, train_labels)
preds = model.predict(eval_data)
print(preds)
MSE = mean_squared_error(eval_labels,preds)
print(f'MSE = {MSE}')

Learning rate set to 0.070239
0:	learn: 0.0585400	total: 122ms	remaining: 4m 4s
1:	learn: 0.0552500	total: 179ms	remaining: 2m 59s
2:	learn: 0.0522246	total: 255ms	remaining: 2m 49s
3:	learn: 0.0494268	total: 306ms	remaining: 2m 32s
4:	learn: 0.0468058	total: 360ms	remaining: 2m 23s
5:	learn: 0.0444496	total: 420ms	remaining: 2m 19s
6:	learn: 0.0422969	total: 468ms	remaining: 2m 13s
7:	learn: 0.0403071	total: 516ms	remaining: 2m 8s
8:	learn: 0.0384835	total: 565ms	remaining: 2m 4s
9:	learn: 0.0368079	total: 612ms	remaining: 2m 1s
10:	learn: 0.0352635	total: 667ms	remaining: 2m
11:	learn: 0.0338696	total: 723ms	remaining: 1m 59s
12:	learn: 0.0326249	total: 776ms	remaining: 1m 58s
13:	learn: 0.0314627	total: 824ms	remaining: 1m 56s
14:	learn: 0.0303813	total: 874ms	remaining: 1m 55s
15:	learn: 0.0293694	total: 924ms	remaining: 1m 54s
16:	learn: 0.0284763	total: 973ms	remaining: 1m 53s
17:	learn: 0.0276867	total: 1.02s	remaining: 1m 52s
18:	learn: 0.0269857	total: 1.07s	remaining: 1m 51s


160:	learn: 0.0174121	total: 8.3s	remaining: 1m 34s
161:	learn: 0.0174028	total: 8.35s	remaining: 1m 34s
162:	learn: 0.0173971	total: 8.4s	remaining: 1m 34s
163:	learn: 0.0173885	total: 8.45s	remaining: 1m 34s
164:	learn: 0.0173767	total: 8.5s	remaining: 1m 34s
165:	learn: 0.0173672	total: 8.55s	remaining: 1m 34s
166:	learn: 0.0173489	total: 8.61s	remaining: 1m 34s
167:	learn: 0.0172970	total: 8.66s	remaining: 1m 34s
168:	learn: 0.0172829	total: 8.71s	remaining: 1m 34s
169:	learn: 0.0172753	total: 8.76s	remaining: 1m 34s
170:	learn: 0.0172687	total: 8.81s	remaining: 1m 34s
171:	learn: 0.0172576	total: 8.86s	remaining: 1m 34s
172:	learn: 0.0172515	total: 8.91s	remaining: 1m 34s
173:	learn: 0.0172447	total: 8.97s	remaining: 1m 34s
174:	learn: 0.0172322	total: 9.02s	remaining: 1m 34s
175:	learn: 0.0172252	total: 9.07s	remaining: 1m 33s
176:	learn: 0.0172133	total: 9.12s	remaining: 1m 33s
177:	learn: 0.0171862	total: 9.18s	remaining: 1m 33s
178:	learn: 0.0171804	total: 9.23s	remaining: 1m 

316:	learn: 0.0158819	total: 16.7s	remaining: 1m 28s
317:	learn: 0.0158777	total: 16.7s	remaining: 1m 28s
318:	learn: 0.0158742	total: 16.8s	remaining: 1m 28s
319:	learn: 0.0158687	total: 16.9s	remaining: 1m 28s
320:	learn: 0.0158637	total: 16.9s	remaining: 1m 28s
321:	learn: 0.0158264	total: 17s	remaining: 1m 28s
322:	learn: 0.0158203	total: 17s	remaining: 1m 28s
323:	learn: 0.0157837	total: 17.1s	remaining: 1m 28s
324:	learn: 0.0157806	total: 17.1s	remaining: 1m 28s
325:	learn: 0.0157700	total: 17.2s	remaining: 1m 28s
326:	learn: 0.0157650	total: 17.2s	remaining: 1m 28s
327:	learn: 0.0157574	total: 17.3s	remaining: 1m 28s
328:	learn: 0.0157535	total: 17.3s	remaining: 1m 27s
329:	learn: 0.0157448	total: 17.4s	remaining: 1m 27s
330:	learn: 0.0157382	total: 17.4s	remaining: 1m 27s
331:	learn: 0.0157325	total: 17.5s	remaining: 1m 27s
332:	learn: 0.0157277	total: 17.5s	remaining: 1m 27s
333:	learn: 0.0157245	total: 17.6s	remaining: 1m 27s
334:	learn: 0.0157193	total: 17.7s	remaining: 1m 2

472:	learn: 0.0149668	total: 25s	remaining: 1m 20s
473:	learn: 0.0149646	total: 25.1s	remaining: 1m 20s
474:	learn: 0.0149614	total: 25.1s	remaining: 1m 20s
475:	learn: 0.0149591	total: 25.2s	remaining: 1m 20s
476:	learn: 0.0149561	total: 25.2s	remaining: 1m 20s
477:	learn: 0.0149473	total: 25.3s	remaining: 1m 20s
478:	learn: 0.0149451	total: 25.3s	remaining: 1m 20s
479:	learn: 0.0149401	total: 25.4s	remaining: 1m 20s
480:	learn: 0.0149321	total: 25.4s	remaining: 1m 20s
481:	learn: 0.0149292	total: 25.5s	remaining: 1m 20s
482:	learn: 0.0149239	total: 25.5s	remaining: 1m 20s
483:	learn: 0.0149212	total: 25.6s	remaining: 1m 20s
484:	learn: 0.0149179	total: 25.6s	remaining: 1m 20s
485:	learn: 0.0149153	total: 25.7s	remaining: 1m 20s
486:	learn: 0.0149116	total: 25.8s	remaining: 1m 20s
487:	learn: 0.0149073	total: 25.8s	remaining: 1m 19s
488:	learn: 0.0149049	total: 25.9s	remaining: 1m 19s
489:	learn: 0.0149015	total: 25.9s	remaining: 1m 19s
490:	learn: 0.0148980	total: 26s	remaining: 1m 1

628:	learn: 0.0143989	total: 33.3s	remaining: 1m 12s
629:	learn: 0.0143975	total: 33.4s	remaining: 1m 12s
630:	learn: 0.0143947	total: 33.4s	remaining: 1m 12s
631:	learn: 0.0143900	total: 33.5s	remaining: 1m 12s
632:	learn: 0.0143873	total: 33.5s	remaining: 1m 12s
633:	learn: 0.0143840	total: 33.6s	remaining: 1m 12s
634:	learn: 0.0143812	total: 33.6s	remaining: 1m 12s
635:	learn: 0.0143787	total: 33.7s	remaining: 1m 12s
636:	learn: 0.0143757	total: 33.7s	remaining: 1m 12s
637:	learn: 0.0143729	total: 33.8s	remaining: 1m 12s
638:	learn: 0.0143699	total: 33.8s	remaining: 1m 12s
639:	learn: 0.0143680	total: 33.9s	remaining: 1m 12s
640:	learn: 0.0143660	total: 33.9s	remaining: 1m 11s
641:	learn: 0.0143647	total: 34s	remaining: 1m 11s
642:	learn: 0.0143616	total: 34.1s	remaining: 1m 11s
643:	learn: 0.0143532	total: 34.1s	remaining: 1m 11s
644:	learn: 0.0143514	total: 34.2s	remaining: 1m 11s
645:	learn: 0.0143475	total: 34.2s	remaining: 1m 11s
646:	learn: 0.0143444	total: 34.3s	remaining: 1m

...

1268:	learn: 0.0128407	total: 1m 8s	remaining: 39.7s
1269:	learn: 0.0128367	total: 1m 9s	remaining: 39.7s
1270:	learn: 0.0128356	total: 1m 9s	remaining: 39.6s
1271:	learn: 0.0128347	total: 1m 9s	remaining: 39.6s
1272:	learn: 0.0128332	total: 1m 9s	remaining: 39.5s
1273:	learn: 0.0128309	total: 1m 9s	remaining: 39.5s
1274:	learn: 0.0128286	total: 1m 9s	remaining: 39.4s
1275:	learn: 0.0128262	total: 1m 9s	remaining: 39.4s
1276:	learn: 0.0128243	total: 1m 9s	remaining: 39.3s
1277:	learn: 0.0128227	total: 1m 9s	remaining: 39.3s
1278:	learn: 0.0128214	total: 1m 9s	remaining: 39.2s
1279:	learn: 0.0128201	total: 1m 9s	remaining: 39.2s
1280:	learn: 0.0128175	total: 1m 9s	remaining: 39.1s
1281:	learn: 0.0128154	total: 1m 9s	remaining: 39.1s
1282:	learn: 0.0128143	total: 1m 9s	remaining: 39.1s
1283:	learn: 0.0128117	total: 1m 9s	remaining: 39s
1284:	learn: 0.0128105	total: 1m 10s	remaining: 39s
1285:	learn: 0.0128090	total: 1m 10s	remaining: 38.9s
1286:	learn: 0.0128078	total: 1m 10s	remaining: 

1422:	learn: 0.0125959	total: 1m 17s	remaining: 31.6s
1423:	learn: 0.0125947	total: 1m 17s	remaining: 31.5s
1424:	learn: 0.0125938	total: 1m 18s	remaining: 31.5s
1425:	learn: 0.0125927	total: 1m 18s	remaining: 31.4s
1426:	learn: 0.0125901	total: 1m 18s	remaining: 31.4s
1427:	learn: 0.0125893	total: 1m 18s	remaining: 31.3s
1428:	learn: 0.0125884	total: 1m 18s	remaining: 31.3s
1429:	learn: 0.0125872	total: 1m 18s	remaining: 31.2s
1430:	learn: 0.0125851	total: 1m 18s	remaining: 31.2s
1431:	learn: 0.0125845	total: 1m 18s	remaining: 31.1s
1432:	learn: 0.0125835	total: 1m 18s	remaining: 31.1s
1433:	learn: 0.0125822	total: 1m 18s	remaining: 31s
1434:	learn: 0.0125810	total: 1m 18s	remaining: 31s
1435:	learn: 0.0125795	total: 1m 18s	remaining: 30.9s
1436:	learn: 0.0125780	total: 1m 18s	remaining: 30.9s
1437:	learn: 0.0125760	total: 1m 18s	remaining: 30.8s
1438:	learn: 0.0125749	total: 1m 18s	remaining: 30.8s
1439:	learn: 0.0125733	total: 1m 18s	remaining: 30.7s
1440:	learn: 0.0125723	total: 1m

1575:	learn: 0.0123710	total: 1m 27s	remaining: 23.4s
1576:	learn: 0.0123696	total: 1m 27s	remaining: 23.4s
1577:	learn: 0.0123676	total: 1m 27s	remaining: 23.3s
1578:	learn: 0.0123662	total: 1m 27s	remaining: 23.3s
1579:	learn: 0.0123649	total: 1m 27s	remaining: 23.2s
1580:	learn: 0.0123628	total: 1m 27s	remaining: 23.2s
1581:	learn: 0.0123620	total: 1m 27s	remaining: 23.1s
1582:	learn: 0.0123605	total: 1m 27s	remaining: 23s
1583:	learn: 0.0123599	total: 1m 27s	remaining: 23s
1584:	learn: 0.0123574	total: 1m 27s	remaining: 22.9s
1585:	learn: 0.0123564	total: 1m 27s	remaining: 22.9s
1586:	learn: 0.0123553	total: 1m 27s	remaining: 22.8s
1587:	learn: 0.0123548	total: 1m 27s	remaining: 22.8s
1588:	learn: 0.0123532	total: 1m 27s	remaining: 22.7s
1589:	learn: 0.0123515	total: 1m 27s	remaining: 22.7s
1590:	learn: 0.0123500	total: 1m 27s	remaining: 22.6s
1591:	learn: 0.0123487	total: 1m 27s	remaining: 22.6s
1592:	learn: 0.0123470	total: 1m 28s	remaining: 22.5s
1593:	learn: 0.0123458	total: 1m

1731:	learn: 0.0121758	total: 1m 35s	remaining: 14.9s
1732:	learn: 0.0121747	total: 1m 36s	remaining: 14.8s
1733:	learn: 0.0121721	total: 1m 36s	remaining: 14.7s
1734:	learn: 0.0121713	total: 1m 36s	remaining: 14.7s
1735:	learn: 0.0121700	total: 1m 36s	remaining: 14.6s
1736:	learn: 0.0121691	total: 1m 36s	remaining: 14.6s
1737:	learn: 0.0121680	total: 1m 36s	remaining: 14.5s
1738:	learn: 0.0121663	total: 1m 36s	remaining: 14.5s
1739:	learn: 0.0121650	total: 1m 36s	remaining: 14.4s
1740:	learn: 0.0121632	total: 1m 36s	remaining: 14.4s
1741:	learn: 0.0121626	total: 1m 36s	remaining: 14.3s
1742:	learn: 0.0121599	total: 1m 36s	remaining: 14.2s
1743:	learn: 0.0121591	total: 1m 36s	remaining: 14.2s
1744:	learn: 0.0121579	total: 1m 36s	remaining: 14.1s
1745:	learn: 0.0121570	total: 1m 36s	remaining: 14.1s
1746:	learn: 0.0121527	total: 1m 36s	remaining: 14s
1747:	learn: 0.0121516	total: 1m 36s	remaining: 14s
1748:	learn: 0.0121502	total: 1m 36s	remaining: 13.9s
1749:	learn: 0.0121494	total: 1m

1887:	learn: 0.0119793	total: 1m 44s	remaining: 6.2s
1888:	learn: 0.0119783	total: 1m 44s	remaining: 6.15s
1889:	learn: 0.0119776	total: 1m 44s	remaining: 6.09s
1890:	learn: 0.0119760	total: 1m 44s	remaining: 6.04s
1891:	learn: 0.0119750	total: 1m 44s	remaining: 5.98s
1892:	learn: 0.0119743	total: 1m 44s	remaining: 5.93s
1893:	learn: 0.0119734	total: 1m 44s	remaining: 5.87s
1894:	learn: 0.0119725	total: 1m 44s	remaining: 5.82s
1895:	learn: 0.0119708	total: 1m 45s	remaining: 5.76s
1896:	learn: 0.0119693	total: 1m 45s	remaining: 5.71s
1897:	learn: 0.0119685	total: 1m 45s	remaining: 5.65s
1898:	learn: 0.0119677	total: 1m 45s	remaining: 5.59s
1899:	learn: 0.0119665	total: 1m 45s	remaining: 5.54s
1900:	learn: 0.0119651	total: 1m 45s	remaining: 5.48s
1901:	learn: 0.0119633	total: 1m 45s	remaining: 5.43s
1902:	learn: 0.0119625	total: 1m 45s	remaining: 5.37s
1903:	learn: 0.0119618	total: 1m 45s	remaining: 5.32s
1904:	learn: 0.0119611	total: 1m 45s	remaining: 5.26s
1905:	learn: 0.0119592	total:

In [109]:
eval_labels

66960    0.304000
66961    0.315625
66962    0.314875
66963    0.323875
66964    0.321625
           ...   
89275    0.282375
89276    0.274875
89277    0.244500
89278    0.260250
89279    0.318000
Name: pressure, Length: 22320, dtype: float64

In [34]:
print(train1[used_feat].describe())
print(train1[used_feat].shape)
print(train1[used_feat].isnull().sum())
print(train1[used_feat].info())

NameError: name 'train1' is not defined

         Unnamed: 0      MeasName           Hour            Day      Year  \
count  525600.00000  525600.00000  525600.000000  525600.000000  525600.0   
mean   262799.50000      15.50000      11.500000      15.720548    2018.5   
std    151727.79508       8.65545       6.922193       8.796255       0.5   
min         0.00000       1.00000       0.000000       1.000000    2018.0   
25%    131399.75000       8.00000       5.750000       8.000000    2018.0   
50%    262799.50000      15.50000      11.500000      16.000000    2018.5   
75%    394199.25000      23.00000      17.250000      23.000000    2019.0   
max    525599.00000      30.00000      23.000000      31.000000    2019.0   

               Month  log_1_pressure  log_2_pressure  log_3_pressure  \
count  525600.000000   524880.000000   524160.000000   523440.000000   
mean        6.526027        0.250933        0.250900        0.250879   
std         3.447855        0.062097        0.062082        0.062067   
min         1.0000

In [35]:
print(train2020)

       Unnamed: 0        Time  MeasName  Hour  pressure  Time_time  Day  Year  \
0               0  2020-01-01         1     0  0.309625 2020-01-01    1  2020   
1               1  2020-01-01         1     1  0.325750 2020-01-01    1  2020   
2               2  2020-01-01         1     2  0.332875 2020-01-01    1  2020   
3               3  2020-01-01         1     3  0.324250 2020-01-01    1  2020   
4               4  2020-01-01         1     4  0.331750 2020-01-01    1  2020   
...           ...         ...       ...   ...       ...        ...  ...   ...   
89275       89131  2020-08-31        30    19  0.282375 2020-08-31   31  2020   
89276       89132  2020-08-31        30    20  0.274875 2020-08-31   31  2020   
89277       89133  2020-08-31        30    21  0.244500 2020-08-31   31  2020   
89278       89134  2020-08-31        30    22  0.260250 2020-08-31   31  2020   
89279       89135  2020-08-31        30    23  0.318000 2020-08-31   31  2020   

       Month  log_1_pressur

In [372]:
print(train2018)
print(train2019)
print(train2020)

              Time  MeasName  Hour  pressure  Time_time  Day  Year  Month  \
0       2018-01-01         1     0  0.288625 2018-01-01    1  2018      1   
1       2018-01-01         1     1  0.292000 2018-01-01    1  2018      1   
2       2018-01-01         1     2  0.290500 2018-01-01    1  2018      1   
3       2018-01-01         1     3  0.299500 2018-01-01    1  2018      1   
4       2018-01-01         1     4  0.300250 2018-01-01    1  2018      1   
...            ...       ...   ...       ...        ...  ...   ...    ...   
262651  2018-12-31        30    19  0.195000 2018-12-31   31  2018     12   
262652  2018-12-31        30    20  0.187875 2018-12-31   31  2018     12   
262653  2018-12-31        30    21  0.184875 2018-12-31   31  2018     12   
262654  2018-12-31        30    22  0.201375 2018-12-31   31  2018     12   
262655  2018-12-31        30    23  0.212625 2018-12-31   31  2018     12   

        log_1_pressure  log_2_pressure  log_3_pressure  
0                 

In [466]:
print(type(preds))

<class 'numpy.ndarray'>


In [467]:
preds.shape

(22320,)

In [469]:
predsAug = pd.DataFrame(preds, columns = ['Aug'])

In [470]:
print(predsAug)

            Aug
0      0.130150
1      0.136715
2      0.137419
3      0.136765
4      0.134880
...         ...
22315  0.287673
22316  0.281284
22317  0.264417
22318  0.268157
22319  0.297003

[22320 rows x 1 columns]


In [471]:
train1.describe()

,MeasName,Hour,pressure,Day,Year,Month,log_1_pressure,log_2_pressure,log_3_pressure,weather,wind_dir,wind_level
count,22320.000000,22320.000000,22320.000000,22320.000000,22320.0,22320.0,21600.000000,20880.000000,20160.000000,22320.000000,22320.000000,22320.000000
mean,15.500000,11.500000,0.266826,16.000000,2020.0,8.0,0.267247,0.267483,0.267739,1.451613,2.225806,0.677419
std,8.655635,6.922342,0.054823,8.944472,0.0,0.0,0.054690,0.054740,0.054725,0.796733,0.869193,0.467474
min,1.000000,0.000000,-0.006375,1.000000,2020.0,8.0,-0.006375,-0.006375,-0.006375,0.000000,0.000000,0.000000
25%,8.000000,5.750000,0.232500,8.000000,2020.0,8.0,0.232875,0.233125,0.233250,1.000000,2.000000,0.000000
50%,15.500000,11.500000,0.261375,16.000000,2020.0,8.0,0.261750,0.261750,0.262125,2.000000,2.000000,1.000000
75%,23.000000,17.250000,0.298125,24.000000,2020.0,8.0,0.298500,0.298875,0.299031,2.000000,3.000000,1.000000
max,30.000000,23.000000,0.462750,31.000000,2020.0,8.0,0.462750,0.462750,0.462750,2.000000,3.000000,1.000000


In [472]:
train1

,Time,MeasName,Hour,pressure,Time_time,Day,Year,Month,log_1_pressure,log_2_pressure,log_3_pressure,weather,temp_max,temp_min,wind_dir,wind_level
66960,2020-08-01,1,0,0.304000,2020-08-01,1,2020,8,NaN,NaN,NaN,2,24,32,2,0
66961,2020-08-01,1,1,0.315625,2020-08-01,1,2020,8,NaN,NaN,NaN,2,24,32,2,0
66962,2020-08-01,1,2,0.314875,2020-08-01,1,2020,8,NaN,NaN,NaN,2,24,32,2,0
66963,2020-08-01,1,3,0.323875,2020-08-01,1,2020,8,NaN,NaN,NaN,2,24,32,2,0
66964,2020-08-01,1,4,0.321625,2020-08-01,1,2020,8,NaN,NaN,NaN,2,24,32,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89275,2020-08-31,30,19,0.282375,2020-08-31,31,2020,8,0.295500,0.306750,0.309375,1,23,31,3,1
89276,2020-08-31,30,20,0.274875,2020-08-31,31,2020,8,0.288750,0.306375,0.301500,1,23,31,3,1
89277,2020-08-31,30,21,0.244500,2020-08-31,31,2020,8,0.273000,0.281625,0.286500,1,23,31,3,1
89278,2020-08-31,30,22,0.260250,2020-08-31,31,2020,8,0.284250,0.269625,0.268500,1,23,31,3,1


In [473]:
test

,id,Time,MeasName,Hour,Time_time,Day,Year,Month
0,0,2020-02-03,4,0,2020-02-03,3,2020,2
1,1,2020-02-03,4,1,2020-02-03,3,2020,2
2,2,2020-02-03,4,2,2020-02-03,3,2020,2
3,3,2020-02-03,4,3,2020-02-03,3,2020,2
4,4,2020-02-03,4,4,2020-02-03,3,2020,2
...,...,...,...,...,...,...,...,...
40315,40315,2020-09-20,6,19,2020-09-20,20,2020,9
40316,40316,2020-09-20,6,20,2020-09-20,20,2020,9
40317,40317,2020-09-20,6,21,2020-09-20,20,2020,9
40318,40318,2020-09-20,6,22,2020-09-20,20,2020,9


In [474]:
print(train1['log_1_pressure'].isnull().sum())


720


In [33]:
train1819.to_csv('/Users/liruizhi/Desktop/outcome.csv')

In [35]:
train2020.to_csv('/Users/liruizhi/Desktop/testinput.csv')


In [31]:
result181920.to_csv('/Users/liruizhi/desktop/Bigone.csv')